# Agnostic methods for liquid handling


## Basic imports

In [ ]:
#!pip install pylabrobot

In [ ]:
# Basic imports:
from pprint import pprint, pformat
# - liquid handler
from pylabrobot.liquid_handling import LiquidHandler
# - backend
from pylabrobot.liquid_handling.backends import STAR
from pylabrobot.liquid_handling.backends.chatterbox_backend import ChatterBoxBackend
# - deck
from pylabrobot.resources.hamilton import STARLetDeck

## Writing a custom Backend

There are a bunch of useful imports that must come first, and then the definition of a Python class inheriting from "LiquidHandlerBackend".

The new class, as shown below, has placeholders for each of the important methods that the backend must implement.

This new "EchoBackend" mimics the ChatterBox backend already available in PLR. This one saves the objects it received for inspection.

In [ ]:
from typing import List
from pylabrobot.liquid_handling.backends import LiquidHandlerBackend
from pylabrobot.resources import Resource
from pylabrobot.liquid_handling.standard import (
    Pickup,
    PickupTipRack,
    Drop,
    DropTipRack,
    Aspiration,
    AspirationPlate,
    Dispense,
    DispensePlate,
    Move
)


class EchoBackend(LiquidHandlerBackend):
    """ Yet another Chatter box backend for 'How to Open Source' """

    commands = []
    """Just a list to store incoming data, forlater inspection"""

    def __init__(self, num_channels: int = 1, name="echo"):
        """Init method for the EchoBackend."""
        print(f"EchoBackend - Instantiating the EchoBackend with num_channels={num_channels}")
        self.name=name
        super().__init__()
        self._num_channels = num_channels

    async def setup(self):
        await super().setup()
        print("EchoBackend - Setting up the robot.")

    async def stop(self):
        await super().stop()
        print("EchoBackend - Stopping the robot.")

    @property
    def num_channels(self) -> int:
        return self._num_channels

    async def assigned_resource_callback(self, resource: Resource):
        print(f"EchoBackend - Resource {resource.name} was assigned to the robot: " + pformat(resource.serialize))

    async def unassigned_resource_callback(self, name: str):
        print(f"EchoBackend - Resource with name '{name}' was unassigned from the robot.")

    # Atomic implemented in hardware.
    async def pick_up_tips(self, ops: List[Pickup], use_channels: List[int], **backend_kwargs):
        print(f"EchoBackend - {len(self.commands)} - Picking up tips {ops}.")
        self.commands.append({"cmd": "pick_up_tips", "ops": ops, "use_channels": use_channels, **backend_kwargs})

    async def drop_tips(self, ops: List[Drop], use_channels: List[int], **backend_kwargs):
        print(f"EchoBackend - {len(self.commands)} - Dropping tips {ops}.")
        self.commands.append({"cmd": "drop_tips", "ops": ops, "use_channels": use_channels, **backend_kwargs})

    async def aspirate(self, ops: List[Aspiration], use_channels: List[int], **backend_kwargs):
        print(f"EchoBackend - {len(self.commands)} - Aspirating {ops}.")
        self.commands.append({"cmd": "aspirate", "ops": ops, "use_channels": use_channels, **backend_kwargs})

    async def dispense(self, ops: List[Dispense], use_channels: List[int], **backend_kwargs):
        print(f"EchoBackend - {len(self.commands)} - Dispensing {ops}.")
        self.commands.append({"cmd": "dispense", "ops": ops, "use_channels": use_channels, **backend_kwargs})

    # Atomic actions not implemented in hardware.
    async def pick_up_tips96(self, pickup: PickupTipRack, **backend_kwargs):
        raise NotImplementedError("EchoBackend - The backend does not support the CoRe 96.")

    async def drop_tips96(self, drop: DropTipRack, **backend_kwargs):
        raise NotImplementedError("EchoBackend - The backend does not support the CoRe 96.")

    async def aspirate96(self, aspiration: AspirationPlate):
        raise NotImplementedError("EchoBackend - The backend does not support the CoRe 96.")

    async def dispense96(self, dispense: DispensePlate):
        raise NotImplementedError("EchoBackend - The backend does not support the CoRe 96.")

    async def move_resource(self, move: Move, **backend_kwargs):
        """ Move the specified lid within the robot. """
        raise NotImplementedError("EchoBackend - Moving resources is not implemented yet.")


### Usage example

This new class can be used immediately, as is.

Instantiate the Echo backend, with one channel:

In [ ]:
backend = EchoBackend(num_channels=1)

Create an instance of a "liquid handler" with, for example, a Hamilton deck:

In [ ]:
lh = LiquidHandler(backend=backend, deck=STARLetDeck())

await lh.setup()

While using the "liquid handler" we see some messages from the EchoBackend.

It has received data about some kind of resources assigned to the deck.

Populate the deck with labware, that may be interesting:

In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)

tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01')

lh.deck.assign_child_resource(tip_car, rails=3)

plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

lh.deck.assign_child_resource(plt_car, rails=15)

More of the "resource assigned to the robot" messages. Not much fun so far.

To recap, let's check what the liquid handler has so far:

In [ ]:
lh.summary()

### What's going on at the back?

PLR defines that the backend must implement a few "atomic" actions (e.g. aspirate, dispense, etc.).

Lets pick up a tip.

#### Tips: pick_up_tips

In [ ]:
tiprack = lh.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1"])

Aha! This is interesting...

Now the backend is _supposed_ to pick up tips, using that information in some way. But which way is that?

This dummy backend can be used to "inspect" the objects passed to it, 
and help us learn what to expect when writing a _real_ backend.

Let's use the "commands list" we added to the EchoBackend to investigate how. 

In [ ]:
backend.commands[-1]

It seems we get two pieces of info:

- A list with one "operation", offering tip parameters.
- A list of "channels" to use for the operation.

Let's inspect the operation:

In [ ]:
pick_up_op = backend.commands[-1]["ops"][0]

pprint(pick_up_op)

I'm going to guess that the "resource" in there is the same object as the TipSpot in the TipRack:

In [ ]:
pick_up_op.resource is tiprack["A1"][0]

> Cooooorrrectooooo!

So, we're expected to move the robot to that tip, place it into the requested channels, using some "offset".

Lets inspect the "tip spot" a bit more closely:

In [ ]:
tip_spot = pick_up_op.resource

print(tip_spot)

Ok, but... what can it do?

In [ ]:
dir(tip_spot)

I think `get_absolute_location` looks interesting.

Perhaps we can get the absolute spatial coordinate of the tip's location:

In [ ]:
coordinate = tip_spot.get_absolute_location()

print(coordinate)

Yep, seems important. But what does the Z coordinate mean? Is it the "tip" or the "top"?

Or is it something else?

The `tip` object in the operation may shed some light here:

In [ ]:
pick_up_op.tip

Surprisingly, it is not a "PLR resource". It does have, however, two relevant attributes:

- fitting depth: which I figure is the distance that the pipette inserts istelf into the tip from the top side in order to make the seal.
- total length: well, that.

The only variable here is how the Z coordinate of the tip spot relates to the tip.

Before I move on, lets look at the "offset" in the pick up operation.


In [ ]:
# Prints "Default" which only indicates that this should be replaced by a default value.
print(pick_up_op.offset)

Nothing apparently.

FYI, the source states that the offsets are a:

> List of offsets for each channel, a translation that will be applied to the tip drop location. If `None`, no offset will be applied.

The offsets in the list are expected to be `Coordinate` objects.

<!-- This seems to no longer apply:

The offset attribute, unless altered by the lh or backend, is a "Default" object.

It has no information whatsoever, other than hinting that it should be replaced by a default value.

```
from pylabrobot.default import get_value
from pylabrobot.resources.coordinate import Coordinate

origin = Coordinate(0,0,0)

# Prints "Default" which only indicates that this should be replaced by a default value.
print(pick_up_op.offset)

# A default value can be set like this:
default_offset = get_value(value=pick_up_op.offset, default=origin)
print(default_offset)
```
-->

#### Well: aspirate

Aspirate from a well:

In [ ]:
plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1"], vols=[100.0])

Inspect the "Aspiration" operation:

In [ ]:
# Get the operation
ops = lh.backend.commands[1]["ops"]
op = ops[0]

pprint(op)


In [ ]:
# Some useful parameters
op.volume
op.resource.get_absolute_location()
op.blow_out_air_volume
op.liquid_height
op.flow_rate

# The current volume is harder to access, this is defined in the "Container" class.
op.resource.tracker.get_used_volume()
#pprint(op.resource.tracker.serialize())

pprint(op.resource.serialize())

Aspirate from multiple wells, with a single-channel pipette:

In [ ]:
print(f"Backend '{backend.name}' with {backend.num_channels} channel(s).")

try:
  # Aspirate 2.0 uL from A1, and then 3.9 uL from A2?
  await lh.aspirate(plate["A1:A2"], vols=[2.0, 3.0])
except Exception as e:
  print(e)

try:
  # Aspirate 2.0 uL from each well?
  await lh.aspirate(plate["A1:A2"], 2.0)
except Exception as e:
  print(e)

Dispense to another well, and drop the tips (back in the box):

In [ ]:
await lh.dispense(plate["D1"], vols=[100.0])

There are THREE ways to remove tips from the pipettes:

- `return_tips`: "_Return all tips that are currently picked up to their original place_". Takes no arguments.
  - Do not use this if your pipettes cannot eject tips by themselves (i.e. they must go to a certain spot to discard tips, and cannot return tips to their box).
- `discard_tips`: "_Permanently discard tips_", from the specified channels. Uses the deck's "trash area".
- `drop_tips`: "_Drop tips to a resource_". The lowest-level form of this action, used by `return_tips` and `discard_tips`. Requires a list of "tip spots" (undocumented parameter); probably a "_List of tip spots to pick up tips from_".

In [ ]:
await lh.drop_tips(tip_spots=tiprack["A1"])

Inspect the "Dispense" operation:

In [ ]:
# Get the operation
ops = lh.backend.commands[2]["ops"]
op = ops[0]

# Some useful parameters
op.volume
op.resource.get_absolute_location()
op.blow_out_air_volume
op.liquid_height
op.flow_rate

# The current volume is harder to access, this is defined in the "Container" class.
op.resource.tracker.get_used_volume()

# Print everything
pprint(op)
pprint(op.resource.serialize())

Terminate the liquid handler:

In [ ]:
await lh.stop()

#### Number of channels

The number of "pipetting channels" represents how many tips a robot/backend/tool can operate simultaneously.

This name is derived from the name of "multichannel" micropipettes, which can tipically fit 8 or 12 individual tips in a row, and load liquid into them simultaneously.

Advanced robotic pipettes may support multi-channel with variable per-tip volume control (which is more alike a multi-pipette tool).

In [ ]:
backend = EchoBackend(num_channels=2)

Setup the backend and populate the deck:

In [ ]:
lh = LiquidHandler(backend=backend, deck=STARLetDeck())
await lh.setup()

tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01')

lh.deck.assign_child_resource(tip_car, rails=3)

plt_car = PLT_CAR_L5AC_A00(name='plate carrier 1')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')
plt_car[1] = Cos_96_DW_1mL(name='plate_02')

lh.deck.assign_child_resource(plt_car, rails=15)

lh.summary()

Inspect the tip rack:

In [ ]:
# plate.children[0].serialize()
tiprack.serialize()

Pickup tips and aspirate some liquid:

In [ ]:
tiprack = lh.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1:B1"])

plate1 = lh.get_resource("plate_01")
plate2 = lh.get_resource("plate_02")
await lh.aspirate(plate1["A1"] + plate2["A2"], vols=100.0, use_channels=[0, 1])

#### Miss-use of channels in PLR

Now lets try to break PLR by placing more tips than the backend supports:

In [ ]:
import traceback

try:
  tiprack = lh.get_resource("tips_01")
  await lh.pick_up_tips(tiprack["A1:C1"])
except:
  print(tracebcack.format_exc())


It is sensible that it fails, see discussion at: https://github.com/PyLabRobot/pylabrobot/issues/26

Stop the backend with the `stop` method. Say bye!

In [ ]:
await lh.stop()

### Multiple tools

Using PLR with multiple-tool machines.

In [ ]:
# ?

It is probably best to leave this to the backend.

# Pipetting-bot backend

The notes above are generic. The following is specific to a custom pipetting robot, based on the firmware Klipper (and its friends).

The idea here was to write a "thin" backend, which delegates the most important functions to two custom modules: `newt` and `piper`.

- `piper` parses json-structured commands, generates GCODE for the robot, and sends it.
- `newt` generates valid json objects for `piper`, relying on `jsonschema` and the data provided by PLR.

## Setting up

This setup is optional, and specific to the development of the `piper` backend.

It may serve as an example setup for setting up the python modules used by your backend.

### Virtual environment

You can create a virtual environment and test everything from there.

Run the following to load your virtual environment and update the modules. Replace the paths below to match yours:

```bash
source .venv/bin/activate
pip install git+https://gitlab.com/pipettin-bot/pipettin-piper.git
pip install pylabrobot[dev]
```

Run those commands each time you want to test out changes.

You can then check if the correct environment is in use:

In [ ]:
!echo $VIRTUAL_ENV

import os, sys

print("Current working directory: " + os.getcwd())
print("Using python: " + sys.executable)


Install `piper` from git:

In [ ]:
# Install dependencies
!pip install git+https://gitlab.com/pipettin-bot/pipettin-piper.git

This will also have installed `newt`, an important dependency to link `piper` with other programs.

In [ ]:
import piper, newt

## The piper backend

Write a new backend that uses the "piper" GCODE-generating module from the pipetting-bot project.

TODO:

- [ ] Bug: commands can be sent to the backend before it is ready.
- [ ] Implement waiting/checking of command responses, and proper blocking.
- [ ] ¿Use queues?
- [ ] Write adapter to use decks as piper workspaces.
  - This may not be necessary if all the required info is passed to the PLR.
  - ¿Do other backends need to access the "deck" object?
- [ ] Write adapter to pass object coordinates from PLR to piper (not relying on workspaces).
  - See pylabrobot/liquid_handling/standard.py
- [ ] A lot more...

In [ ]:
# Install the "piper" module from the path to the module's directory (replace appropriately).
# $ pip install -e ~/Projects/robot/pipettin-bot/klipper/piper/

# Alternatively install the packages into the current environment:
# %pip install websockets pymongo aiohttp python-socketio

In [ ]:
# Append custom module paths.
# import os, sys
# module_path = os.path.expanduser("~/Projects/GOSH/gosh-col-dev/pipettin-grbl/klipper/code/")
# sys.path.append(module_path)

from pprint import pprint
from pylabrobot.liquid_handling.backends.piper_backend import PiperBackend

Check that mongo is accessible from outside (if running this remotely). Otherwise see: https://www.digitalocean.com/community/tutorials/how-to-configure-remote-access-for-mongodb-on-ubuntu-20-04

In [ ]:
## !nc -zv 192.168.11.39 27017  # Should output "192.168.11.39 27017 (mongodb) open"
!nc -zv localhost 27017  # Should output "localhost [127.0.0.1] 27017 (mongodb) open"

The following `piper` backend can be used in "dry" mode, which does not require active connections to the rest of the software associated to the Pipettin-bot.

In [ ]:
# Set "mongo_url = None" to skip the database connection.
# backend = PiperBackend(verbose=False,
#                        mongo_url=None, # "mongodb://192.168.11.39:27017/",
#                        sio_address = "http://192.168.11.39:3333", # Pipettin GUI node server.
#                        ws_address = "ws://192.168.11.39:7125/websocket", # Moonraker server.
# )

backend = PiperBackend(verbose=False, dry=True, clearance=80.0,
                       mongo_url=None, # "mongodb://192.168.11.39:27017/",
                       sio_address = "http://localhost:3333", # Pipettin GUI node server.
                       ws_address = "ws://localhost:7125/websocket", # Moonraker server.
)

### Setup method

You can setup the backend independently of a PLR liquid handler object.

In [ ]:
# backend.moon.start_as_task()
# await backend.moon.wait_for_setup(timeout=2.0, raise_error=True)
# await backend.moon.wait_for_ready(reset=True, wait_time=1.1, timeout=10.0)

# await backend.setup()

# pprint(backend.tracker["PLR setup"], width=120)
# backend.moon.get_response_by_id(cmd_id="PLR setup")

### Deck and LiquidHandler setup


Setup the liquidhandler class. This also calls `setup` in the backend.

In [ ]:
# - liquid handler
from pylabrobot.liquid_handling import LiquidHandler

lh = LiquidHandler(backend=backend, deck=STARLetDeck())

await lh.setup()

There should be a "homing" action in the piper backend.

In [ ]:
backend.builder.current_action

A PLR `deck` is required. We'll use the default for now.

In [ ]:
# - deck
from pylabrobot.resources.hamilton import STARLetDeck
# - resources
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)

tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01')

lh.deck.assign_child_resource(tip_car, rails=3)

plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

lh.deck.assign_child_resource(plt_car, rails=15)

# lh.summary()

### Pick tips

In [ ]:
tiprack = lh.get_resource("tips_01")

await lh.pick_up_tips(tiprack["A1"])

In [ ]:
backend.builder.current_action

### Pipetting

In [ ]:
plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1"], vols=[100.0])

In [ ]:
backend.builder.current_action

In [ ]:
await lh.dispense(plate["A2"], vols=[50.0])

In [ ]:
backend.builder.current_action

### Drop tips

In [ ]:
await lh.discard_tips()

In [ ]:
backend.builder.current_action

### Cleanup

In [ ]:
await lh.stop()

In [ ]:
backend.builder.current_action

## Internal backend commands

Examine the backend's status:

In [ ]:
pprint(backend.moon.status())

Play with lower-level backend commands:

In [ ]:
await backend.moon.wait_for_idle_printer(timeout = 20.0)

In [ ]:
cmd_id = await backend.moon.send_gcode_cmd("SET_KINEMATIC_POSITION X=0 Y=0 Z=0", wait=True, check=True, timeout=1.0)
pprint(backend.tracker[cmd_id])

In [ ]:
cmd_id = await backend.moon.send_gcode_cmd("FICTITIOUS COMMAND", wait=True, check=True, timeout=10.0)
# cmd_id = await backend.moon.send_gcode_cmd("M105", wait=True, check=True, timeout=10.0)
#cmd_id = await backend.moon.send_gcode_cmd("G0 X1", wait=True, check=True, timeout=10.0)
pprint(backend.tracker[cmd_id], width=120)

In [ ]:
# pprint(backend.tracker[cmd_id])
# backend.tracker[cmd_id]["response"]
# backend.tracker[cmd_id]["response"]["error"]
# backend.tracker[cmd_id]["response"]["error"]["message"]
# print(backend.tracker[cmd_id]["response"]["error"]["message"])

Make GCODE for an action:

In [ ]:
# cmd_id = await backend.moon.send_gcode_cmd("G0 X100", wait=True, check=True, timeout=1.0)
# pprint(backend.tracker[cmd_id])


# Hardcode the heigh clearance
backend.builder.clearance = 100.0

# TODO: customize parameters.
home_action = {'cmd': 'HOME'}
pick_up_tip_action = {
    'args': {'item': '200ul_tip_rack_MULTITOOL 1', 'tool': 'P200'},
    'cmd': 'PICK_TIP'}

# Platformless tip pickup
minimal_tip_data = {'maxVolume': 160, 'tipLength': 50.0, 'volume': 0}
pick_up_tip_action_coords = {
    'args': {'coords': {"x": 20, "y": 200, "z": 30},
             'tool': 'P200',
             'tip': minimal_tip_data},
    'cmd': 'PICK_TIP'}

# Make GCODE
action = home_action
gcode = backend.builder.parseAction(action=action)
pprint(gcode)

In [ ]:
# Send commands.
cmd_id = await backend.moon.send_gcode_script(gcode, wait=True, check=True, timeout=2.0)

In [ ]:
# pprint(backend.tracker[cmd_id])
# backend.tracker[cmd_id]["response"]
# backend.tracker[cmd_id]["response"]["error"]
# print(backend.tracker[cmd_id]["response"]["error"]["message"])

In [ ]:
# Turn steppers off
# cmd_id = await backend.moon.send_gcode_cmd("M84", wait=True, check=True, timeout=2.0)
# pprint(backend.tracker[cmd_id], width=120)

In [ ]:
# Wait for idle printer.
result = await backend.moon.wait_for_idle_printer(timeout = 20.0)

print(result)

In [ ]:
await backend.stop()

## Deck setup examples

> Pipettin's deck has not been implemented yet. Default PLR examples follow.

### Setup the Liquid Handler

Create a new liquid handler using `STAR` as its backend, and the hamilton "deck".

In [ ]:
# Basic imports:
from pprint import pprint, pformat
# - liquid handler
from pylabrobot.liquid_handling import LiquidHandler
# - backend
from pylabrobot.liquid_handling.backends.piper_backend import PiperBackend
# - deck
from pylabrobot.resources.hamilton import STARLetDeck

backend = PiperBackend(verbose=False,
                       mongo_url=None, # "mongodb://192.168.11.39:27017/",
                       sio_address = "http://localhost:3333", # Pipettin GUI node server.
                       ws_address = "ws://localhost:7125/websocket", # Moonraker server.
                       dry = True, # Actions have no consequence.
                       clearance=100.0 # TODO: get this from the deck's definition.
)

lh = LiquidHandler(backend=backend, deck=STARLetDeck())

The final step is to open communication with the robot. This is done using the {func}`~pylabrobot.liquid_handling.LiquidHandler.setup` method.

In [ ]:
await lh.setup()

### Define the deck's contents


Now that we have a `LiquidHandler` instance, we can define the deck layout.

The layout in this tutorial will contain five sets of standard volume tips with filter, 1 set of 96 1mL wells, and tip and plate carriers on which these resources are positioned.

Start by importing the relevant objects and variables from the PyHamilton package. This notebook uses the following resources:

- {class}`~pylabrobot.resources.ml_star.tip_carriers.TIP_CAR_480_A00` tip carrier
- {class}`~pylabrobot.resources.ml_star.plate_carriers.PLT_CAR_L5AC_A00` plate carrier
- {class}`~pylabrobot.resources.corning_costar.plates.Cos_96_DW_1mL` wells
- {class}`~pylabrobot.resources.ml_star.tip_racks.HTF_L` tips

In [ ]:
from pylabrobot.resources import (
    TIP_CAR_480_A00,
    PLT_CAR_L5AC_A00,
    Cos_96_DW_1mL,
    HTF_L
)

Then create a tip carrier named `tip carrier`, which will contain tip rack at all 5 positions. These positions can be accessed using `tip_car[x]`, and are 0 indexed.

In [ ]:
tip_car = TIP_CAR_480_A00(name='tip carrier')
tip_car[0] = HTF_L(name='tips_01')

Use {func}`~pylabrobot.resources.abstract.assign_child_resources` to assign the tip carrier to the deck of the liquid handler. All resources contained by this carrier will be assigned automatically.

In the `rails` parameter, we can pass the location of the tip carrier. The locations of the tips will automatically be calculated.

In [ ]:
lh.deck.assign_child_resource(tip_car, rails=3)

Repeat this for the plates.

In [ ]:
plt_car = PLT_CAR_L5AC_A00(name='plate carrier')
plt_car[0] = Cos_96_DW_1mL(name='plate_01')

In [ ]:
lh.deck.assign_child_resource(plt_car, rails=15)

Let's look at a summary of the deck layout using {func}`~pylabrobot.liquid_handling.LiquidHandler.summary`.

In [ ]:
lh.summary()

## Protocol actions

> Default examples from PLR.

### Picking up tips

Picking up tips is as easy as querying the tips from the tiprack.

In [ ]:
tiprack = lh.get_resource("tips_01")
await lh.pick_up_tips(tiprack["A1"])

### Aspirating and dispensing

Aspirating and dispensing work similarly to picking up tips: where you use booleans to specify which tips to pick up, with aspiration and dispensing you use floats to specify the volume to aspirate or dispense in $\mu L$.

The cells below move liquid from wells `'A1:C1'` to `'D1:F1'` using channels 1, 2, and 3 using the {func}`~pylabrobot.liquid_handling.LiquidHandler.aspirate` and {func}`~pylabrobot.liquid_handling.LiquidHandler.dispense` methods.

In [ ]:
plate = lh.get_resource("plate_01")
await lh.aspirate(plate["A1"], vols=[100.0])

After the liquid has been aspirated, dispense it in the wells below. Note that while we specify different wells, we are still using the same channels. This is needed because only these channels contain liquid, of course.

In [ ]:
await lh.dispense(plate["D1"], vols=[100.0])

Let's move the liquid back to the original wells.

In [ ]:
await lh.aspirate(plate["D1"], vols=[100.0])
await lh.dispense(plate["A1"], vols=[100.0])

### Discarding tips

Finally, you can discard tips by using the {func}`~pylabrobot.liquid_handling.LiquidHandler.discard_tips` method.

In [ ]:
await lh.discard_tips()

### Cleaning up

In [ ]:
await lh.stop()